In [416]:
import csv
import math
import numpy as np
import pandas as pd
import locale

In [417]:
def BlackScholesFD(file_path, K, r, T):
    results = []
    locale.setlocale(locale.LC_ALL, 'en_US.UTF8')
    
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            results.append(row)
            
    S = [None]*(len(results) - 1)
    date = [None]*(len(results) - 1)
    for i in range(1,len(results)):
        S[i-1] = locale.atof(results[i][1].strip(" $"))
        date[i-1] = results[i][0]
    
    S.reverse()
    date.reverse()

    mu = sum(S)/len(S)
    i = 0
    deviations = [None]*len(S)
    
    for i in range(len(S)):
        deviations[i] = (S[i] - mu)**2
    i=0
       
    var = sum(deviations)/len(S)
    SIGMA = math.sqrt(var)/mu
    
    ds = max(S)/96
    dt = 1/(len(date) - 1)
    
    NTS = int(T / dt) + 1
    dt = T / NTS
    
    value_matrix = np.zeros((97, int(NTS)))
    asset_price = np.arange(NAS*ds,-1,-ds)

    value_matrix[:,-1]= np.maximum(asset_price - K,0)

    for x in range(1,NTS):
        value_matrix[-1,-x-1] = value_matrix[-1,-x]* math.exp(-r*dt)
    x=0    

    for x in range(1,NTS):
        for y in range(1,int(96)):
            Delta = (value_matrix[y-1,-x] - value_matrix[y+1,-x]) / 2 / ds
            Gamma = (value_matrix[y-1,-x] - (2 * value_matrix[y,-x]) + value_matrix[y+1,-x]) / ds / ds
            Theta = (-.5 * SIGMA**2 * asset_price[y]**2 * Gamma) - (r * asset_price[y] * Delta) + (r * value_matrix[y,-x])
        
            value_matrix[y,-x-1] = value_matrix[y,-x] - Theta * dt

            value_matrix[0,-x-1] = 2 * value_matrix[1,-x-1] - value_matrix[2,-x-1] 
        
    i = 0
    F = []
    for i in range(len(S)):
        for j in range(len(asset_price)):
            if abs(S[i] - asset_price[j]) < ds/2:
                F.append(round(value_matrix[j][i],2))
            
    np_array = np.transpose([date, S, F])
    new_file = file_path.replace('.csv', 'withcalloptionpricesfromfinitedifference.csv')

    return pd.DataFrame(np_array).to_csv(new_file, header = ["Time", "Stock Price", "Call Option Price from Finite Difference"], index = None)

In [418]:
BlackScholesFD(r"C:\Users\drumm\Documents\call option pricing\nvdiastockprice6months.csv", 190, 0.065, 1)